In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
Train_path = '/content/gdrive/MyDrive/data/dataset/Train'
Val_path = '/content/gdrive/MyDrive/data/dataset/Val'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [ ]:
from keras import metrics
from keras.backend import dropout
#CNN based model in keras
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])



In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 conv2d_13 (Conv2D)          (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 110, 110, 64)     0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 110, 110, 64)      0         
                                                                 
 conv2d_14 (Conv2D)          (None, 108, 108, 64)      36928     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                 

In [ ]:
#train from scratch

train_datagen = image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

val_datagen = image.ImageDataGenerator(
    rescale=1./255,
)

test_datagen = image.ImageDataGenerator(
    rescale=1./255,
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/gdrive/MyDrive/data/dataset/Train',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary')

Found 232 images belonging to 2 classes.


In [ ]:
train_generator.class_indices


{'covid': 0, 'normal': 1}

In [ ]:
val_generator = val_datagen.flow_from_directory(
    '/content/gdrive/MyDrive/data/dataset/Val',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 80 images belonging to 2 classes.


In [ ]:
val_generator.class_indices

{'covid': 0, 'normal': 1}

In [ ]:
test_generator = test_datagen.flow_from_directory(
    '/content/gdrive/MyDrive/data/dataset/Test',
    target_size=(224,224),
    class_mode='binary',
    shuffle=False
)

Found 80 images belonging to 2 classes.


In [ ]:
test_generator.class_indices

{'covid': 0, 'normal': 1}

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch = 8,
    epochs = 10,
    validation_data = val_generator,
    validation_steps = 2
    )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
8/8 [==============================] - 82s 11s/step - loss: 0.9456 - accuracy: 0.4957 - val_loss: 0.6886 - val_accuracy: 0.5312
Epoch 2/10
8/8 [==============================] - 79s 10s/step - loss: 0.6666 - accuracy: 0.6681 - val_loss: 0.6136 - val_accuracy: 0.9844
Epoch 3/10
8/8 [==============================] - 78s 10s/step - loss: 0.5424 - accuracy: 0.7845 - val_loss: 0.4794 - val_accuracy: 0.6562
Epoch 4/10
8/8 [==============================] - 78s 9s/step - loss: 0.5052 - accuracy: 0.7629 - val_loss: 0.3822 - val_accuracy: 0.9688
Epoch 5/10
8/8 [==============================] - 82s 10s/step - loss: 0.4523 - accuracy: 0.8448 - val_loss: 0.3525 - val_accuracy: 0.9844
Epoch 6/10
8/8 [==============================] - 79s 10s/step - loss: 0.3691 - accuracy: 0.8793 - val_loss: 0.2993 - val_accuracy: 0.9844
Epoch 7/10
8/8 [==============================] - 78s 10s/step - loss: 0.3633 - accuracy: 0.8966 - val_loss: 0.2991 - val_accuracy: 0.9688
Epoch 8/10
8/8 [============

In [ ]:
prediction = model.predict_generator(test_generator)
#print(prediction)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


In [ ]:
from sklearn.metrics import confusion_matrix
y_true = np.array([0] * 40 + [1] * 40)
y_pred = prediction > 0.5

confusion_matrix(y_true, y_pred)


array([[40,  0],
       [ 0, 40]])